# Computing the capacity of a cube with a reentrant corner

### Background

The capacity $\text{cap}(\Omega)$ of an isolated conductor $\Omega\subset\mathbb{R}^3$
with boundary $\Gamma:=\partial\Omega$ 
measures its ability to store charges. It is defined 
as the ratio of the total surface equilibrium charge
relative to its surface potential value. To compute the capacity,
we therefore need to solve the following exterior Laplace problem for the equilibrium
potential $u$ with unit surface value:
$$
\begin{align}
-\Delta u &= 0\text{ in }\mathbb{R}^3\backslash\overline{\Omega},\\
u &= 1\text{ on }\Gamma:=\partial\Omega,\\
|u(x)| &=\mathcal{O}\left(|x|^{-1}\right)\text{ as }|x|\rightarrow\infty.
\end{align}
$$
The total surface charge of an isolated conductor is given by Gauss law as
$$
\text{cap}(\Omega)=-\epsilon_0\int_{\Gamma}\frac{\partial u}{\partial\nu}(x)\,d\Gamma(x).
$$
Here, $\nu(x)$ is the outward pointing normal direction for $x\in\Gamma$, and $\epsilon_0$ is the electric constant with value $\epsilon_0\approx 8.854\times 10^{-12}\,{\rm F/m}$. For simplicity, in the following we will leave out this constant and just use the expression $\text{cap}(\Omega)=-\int_{\Gamma}\partial u/\partial\nu\,d\Gamma$.

Using Green's representation theorem and noting that the exterior Laplace double-layer potential is zero for constant densities, we can represent the solution $u$ as
$$
u(x) = -\int_{\Gamma} G(x,y)\phi(y)\,d\Gamma(y) \quad\text{for all }x\in\mathbb{R}^3\backslash\overline{\Omega},
$$
where $\phi:={\partial u}/{\partial\nu}$ is the normal derivative of the exterior solution $u$ and $G(x,y):=\frac{1}{4\pi|x-y|}$ is the  Green's function of the 3D Laplacian. Taking boundary traces we arrive at the boundary integral equation of the first kind
$$
1 = -\int_{\Gamma} G(x,y)\phi(y)\,d\Gamma(y) =: -V\phi(x)\quad\text{for all }x\in\Gamma.
$$
The capacity is now simply given by
$$
\text{cap}(\Omega) = -\int_\Gamma \phi \,d\Gamma.
$$

To improve the convergence we will solve the preconditioned equation
$$
V\tilde{D}\phi = -1,
$$
where $\tilde{D}$ is a regularized hypersingular operator defined by the weak form
$$
\langle \tilde{D}w, v\rangle = \langle Dw, v\rangle + \langle w, 1\rangle\langle v, 1\rangle.
$$

### Implementation

We start with the usual imports.

In [1]:
import bempp.api
import numpy as np

INFO:BEMPP:Dolfin could not be imported. FEM/BEM coupling with FEniCS not available.


The grid is predefined in the shapes module. By default it refines towards the singular corner.

In [2]:
grid = bempp.api.shapes.reentrant_cube(h=0.05, refinement_factor=1)

In the following we define the right-hand side and the operator. We discretize a pair of hypersingular and single-layer operator. Using the corresponding BEM++ function guarantees that only one single-layer operator needs to be discretized. The parameter `dual` means that we discretize the single-layer on the dual grid using piecewise constant basis functions and the hypersingular operator on the original grid using piecewise linear continuous basis functions. The regularization is done via a built-in rank one operator that is added to the hypersingular operator.

In [3]:
def one_fun(x, n, domain_index, res):
    res[0] = -1

slp, hyp, base_slp = bempp.api.operators.boundary.laplace.slp_and_hyp(grid, spaces='dual', return_base_slp=True)  

rank_one_op = bempp.api.RankOneBoundaryOperator(hyp.domain, hyp.range, hyp.dual_to_range)
hyp_regularized = hyp + rank_one_op
    
rhs_fun = bempp.api.GridFunction(slp.range, fun=one_fun)

lhs = slp * hyp_regularized

INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (2876,2876). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.43E-01 sec.


We use GMRES to solve the system. To improve convergence we use a strong discretization that automatically preconditions with the mass matrix.

In [4]:
from scipy.sparse.linalg import gmres

discrete_op = lhs.strong_form()

number_of_iterations = 0
def callback(x):
    global number_of_iterations
    number_of_iterations += 1
    

sol_vec, info = gmres(discrete_op, rhs_fun.coefficients, callback=callback)
sol_fun = hyp_regularized * bempp.api.GridFunction(hyp.domain, coefficients=sol_vec)

print("Number of iterations: {0}".format(number_of_iterations))

print("The capacity is {0}.".format(-sol_fun.integrate()[0,0]))

INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (2876,2876). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.42E-01 sec.
INFO:BEMPP:SLP. START ASSEMBLY. Dim: (103464,103464). Assembly Type: hmat
INFO:BEMPP:SLP. FINISHED ASSEMBLY. Time: 5.51E+01 sec. Mem Size (Mb): 5.89E+02. Compression: 7.21E-03
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (103464,2876). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.69E-01 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (103464,103464). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.36E-01 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (2876,103464). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.33E-01 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (103464,103464). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.38E-01 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (2876,2876). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.31

Number of iterations: 6
The capacity is 8.11057495456.


We now want to compute local residual error estimates for the computed solution. To do this we exploit that we already have a single layer operator on a barycentrically refined grid, because we use opposite order preconditioning with piecewise constant basis functions on the dual grid. We can therefore efficiently evaluate the surface gradient on the barycentrically refined grid.

First, we compute the map from element indices of the original grid to descendent element indices on the barycentric refinement. Hence, ``bary_map[i,:]`` are the indices of the elements in the barycentric refinement that are associated with the element with index i on the original grid. 

In [5]:
bary_map = grid.barycentric_descendents_map()

For the error estimator we want to compute element diameters. This is handled by the following function.

In [6]:
def diameter(element):
    """Compue the diameter of an element."""
    corners = element.geometry.corners
    d0 = np.linalg.norm(corners[:,0] - corners[:,1])
    d1 = np.linalg.norm(corners[:,0] - corners[:, 2])
    d2 = np.linalg.norm(corners[:,1] - corners[:, 2])
    return np.max([d0, d1, d2])

We now evaluate the solution on the barycentrically refined grid. The function space on the barycentric refinement is the space of piecewise linear, discontinuous functions. This was used automatically by BEM++ to assemble the base single-layer operator on the barycentric refinement.

To map the solution we define an identity mapping into this space. 

In [7]:
map_to_base_space = bempp.api.operators.boundary.sparse.identity(sol_fun.space, base_slp.domain, base_slp.domain)
base_sol_fun = map_to_base_space * sol_fun

base_slp_times_sol = base_slp * base_sol_fun

INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (103464,103464). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.48E-01 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (2876,103464). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.31E-01 sec.
INFO:BEMPP:IDENTITY. START ASSEMBLY. Dim: (103464,103464). Assembly Type: sparse
INFO:BEMPP:IDENTITY. FINISHED ASSEMBLY. Time: 1.28E-01 sec.


We now evaluate the squared error estimate $\eta_{\mathcal{T}}^2 = \text{diam}(\mathcal{T})\|\nabla V\phi\|_{L^2(\mathcal{T})}^2$ on each element $\mathcal{T}$ of the barycentric refinement and sum of the contributions of the barycentric triangles associated with each triangle of the original grid. We can ignore the right-hand side function $1$ as it is constant and therefore its surface gradient is zero.

In [8]:
bary_grid = grid.barycentric_grid()

local_errors_squared_bary = np.zeros(bary_grid.leaf_view.entity_count(0), dtype='float64')
local_errors = np.zeros(grid.leaf_view.entity_count(0), dtype='float64')
bary_index_set = bary_grid.leaf_view.index_set()

# Compute the error estimates on the barycentric refinement

for element in bary_grid.leaf_view.entity_iterator(0):
    index = bary_index_set.entity_index(element)
    gradient_norm = base_slp_times_sol.surface_grad_norm(element)
    local_error_squared = diameter(element) * gradient_norm**2
    local_errors_squared_bary[index] = local_error_squared

# Sum up the local contributions for each triangle to obtain an error estimate on the
# original grid.
    
for m in range(bary_map.shape[0]):
    for n in range(bary_map.shape[1]):
        local_errors[m] += local_errors_squared_bary[bary_map[m, n]]

local_errors = np.sqrt(local_errors)
total_error = np.linalg.norm(local_errors)

print("Total H^(-1/2) error estimate for normal derivative: {0}".format(np.linalg.norm(local_errors)))
print("Error estimate for capacity: {0}".format(total_error**2))

Total H^(-1/2) error estimate for normal derivative: 0.0772701782873
Error estimate for capacity: 0.00597068045255


We now want to plot the errors on each element. We first have to reorder the vector with elementwise errors to fit to the indices of a piecewise constant function space. The numbering of the two is not necessarily identical, depending on the underlying grid manager. We therefore reorder according to the correct indexing to make sure that the code works no matter what grid manager BEM++ is compiled with.

In [9]:
const_space = bempp.api.function_space(grid, "DP", 0)

# Resort the error contributions
sorted_local_errors = np.zeros_like(local_errors)
index_set = grid.leaf_view.index_set()
for element in grid.leaf_view.entity_iterator(0):
    index = index_set.entity_index(element)
    global_dof_index = const_space.get_global_dofs(element, dof_weights=False)[0]
    sorted_local_errors[global_dof_index] = local_errors[index]

bempp.api.GridFunction(const_space, coefficients=sorted_local_errors).plot()

The last command should plot the following error picture. <img src="lcube_local_errors.png">